# Sentinel 2 composite using mask_s2_clouds
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lalgonzales/gishndev/blob/main/docs/examples/s2_composite.ipynb)


This notebook show how can you get clean composites using the function, this function apply diferents filters to the image to get a clean composite.

In [ ]:
# !pip install gishndev

In [ ]:
import ee
import geemap
from gishndev import mask_s2_clouds

ee.Authenticate()
ee.Initialize()

In [ ]:
# Sentinel 2 collection
s2_col = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# Google Cloud Score + collection
score_plus_col = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")

# Define the date range and region of interest
start_date = "2020-01-01"
end_date = "2020-12-31"
roi = ee.FeatureCollection("USDOS/LSIB/2017").filter(
    ee.Filter.eq("COUNTRY_NA", "Honduras")
)
ext = roi.geometry().bounds()

# Get a clean collection of Sentinel 2 images
sen2 = (
    s2_col.filterDate(start_date, end_date)
    .filterBounds(ext)
    .linkCollection(score_plus_col, ["cs_cdf"])
    .map(lambda img: img.updateMask(img.select("cs_cdf").gte(0.5)))
    .map(mask_s2_clouds)
)

sen2_median = sen2.median()

In [ ]:
m = geemap.Map()
m.center_object(roi, zoom=8)
m.add_layer(
    sen2_median, {"bands": ["B4", "B3", "B2"], "min": 0, "max": 2000}, "Sentinel 2"
)
m